In [1]:
import numpy as np
from datetime import datetime

# Load features previously extracted from training data

In [2]:
x_train_features = np.load('x_train_features.npy')
y_train = np.load('y_train.npy')

x_test_features = np.load('x_test_features.npy')
y_test = np.load('y_test.npy')

print(x_train_features.shape, y_train.shape, x_test_features.shape, y_test.shape)

(29404, 6272) (29404, 5) (4861, 6272) (4861, 5)


# Prepare new classification model

In [3]:
from pond.tensor import NativeTensor, PrivateEncodedTensor, PublicEncodedTensor
from pond.nn import Dense, Sigmoid, Reveal, Diff, Softmax, CrossEntropy, Sequential, DataLoader

In [4]:
classifier = Sequential([
    Dense(128, 6272),
    Sigmoid(),
    # Dropout(.5),
    Dense(5, 128),
    Reveal(),
    Softmax()
])

In [5]:
def accuracy(classifier, x, y, verbose=0, wrapper=NativeTensor):
    predicted_classes = classifier \
        .predict(DataLoader(x, wrapper), verbose=verbose).reveal() \
        .argmax(axis=1)
        
    correct_classes = NativeTensor(y) \
        .argmax(axis=1)
        
    matches = predicted_classes.unwrap() == correct_classes.unwrap()
    return sum(matches)/len(matches)

# [optional] Fine-tune using NativeTensor to get accuracy expectations

In [22]:
classifier.initialize()

start = datetime.now()
classifier.fit(
    DataLoader(x_train_features, wrapper=NativeTensor), 
    DataLoader(y_train, wrapper=NativeTensor), 
    loss=CrossEntropy(), 
    epochs=5,
    verbose=1,
    learning_rate=.05
)
stop = datetime.now()

print("Elapsed:", stop - start)

2018-01-28 18:06:54.151641 Epoch 0
2018-01-28 18:07:22.623861 Epoch 1
2018-01-28 18:07:50.184193 Epoch 2
2018-01-28 18:08:19.078930 Epoch 3
2018-01-28 18:08:45.699411 Epoch 4
Elapsed: 0:02:18.770608


In [23]:
train_accuracy = accuracy(classifier, x_train_features, y_train)
test_accuracy  = accuracy(classifier, x_test_features,  y_test)

print("Train accuracy:", train_accuracy)
print("Test accuracy:",  test_accuracy)

Train accuracy: 0.938171677323
Test accuracy: 0.936844270726


# [optional] Fine-tune using PublicEncodedTensor to get baseline

In [ ]:
classifier.initialize()

start = datetime.now()
classifier.fit(
    DataLoader(x_train_features, wrapper=PublicEncodedTensor), 
    DataLoader(y_train, wrapper=PublicEncodedTensor),
    loss=CrossEntropy(), 
    epochs=3,
    verbose=1
)
stop = datetime.now()

print("Elapsed:", stop - start)

In [ ]:
train_accuracy = accuracy(classifier, x_train_features, y_train)
test_accuracy  = accuracy(classifier, x_test_features,  y_test)

print("Train accuracy:", train_accuracy)
print("Test accuracy:",  test_accuracy)

# Fine-tune using PrivateEncodedTensor

In [ ]:
classifier.initialize()

start = datetime.now()
classifier.fit(
    DataLoader(x_train_features, wrapper=PrivateEncodedTensor), 
    DataLoader(y_train, wrapper=PrivateEncodedTensor),
    loss=CrossEntropy(), 
    epochs=3,
    verbose=1
)
stop = datetime.now()

print("Elapsed:", stop - start)

## Save weights in case something goes wrong

In [ ]:
np.save('layer0_weights_0.npy', classifier.layers[0].weights.shares0)
np.save('layer0_weights_1.npy', classifier.layers[0].weights.shares1)
np.save('layer0_bias_0.npy',    classifier.layers[0].bias.shares0)
np.save('layer0_bias_1.npy',    classifier.layers[0].bias.shares1)

np.save('layer2_weights_0.npy', classifier.layers[2].weights.shares0)
np.save('layer2_weights_1.npy', classifier.layers[2].weights.shares1)
np.save('layer2_bias_0.npy',    classifier.layers[2].bias.shares0)
np.save('layer2_bias_1.npy',    classifier.layers[2].bias.shares1)

In [ ]:
train_accuracy = accuracy(classifier, x_train_features, y_train)
test_accuracy  = accuracy(classifier, x_test_features,  y_test)

print("Train accuracy:", train_accuracy)
print("Test accuracy:", test_accuracy)